In [3]:
import lucem_illud_2020
import dependencies
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from tweepy import Stream
#from Unicode import unidecode
import time
import datetime 
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.tokenize import TweetTokenizer
import demoji
from nltk.corpus import stopwords
from nltk.corpus import words 
from nltk.corpus import wordnet
from collections import Counter
import sys  
import urllib.parse 
import io 
import json 
import os.path 
import os 
#demoji.download_codes()

#Section 1 Scraping

twitter_base_url = 'http://twitter.com/'
bg = "BillGates"
bg_save = 'BillGates.html'
example_text_file = 'sometextfile.txt'
#information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
#example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
#example_docx_save = 'example.docx'



In [4]:
requests.get(twitter_base_url+bg)
bgRequest = requests.get(twitter_base_url+bg)
print(bgRequest.text[:100])

<!DOCTYPE html>
<html lang="en" data-scribe-reduced-action-queue="true">
  <head>
    
    
    
   


In [7]:
dir(bgRequest)
soup = BeautifulSoup(bgRequest.text,'html.parser')
with open('bg_save',mode='w',encoding='utf-8')as f:
    f.write(bgRequest.text)

In [12]:
bgPTags = soup.body.findAll('p')
for pTag in bgPTags[:3]:
    pTag.text
    
Paragraphs  = []
for pTag in bgPTags:
    Paragraphs.append(re.sub(r'\[\d+\]','',pTag.text))
Paragraphs

["We've detected that JavaScript is disabled in your browser. Would you like to proceed to legacy Twitter?",
 'Yes',
 "Sharing things I'm learning through my foundation work and other interests.",
 "Are you sure you want to view these Tweets? Viewing Tweets won't unblock @BillGates",
 "We don't yet have all the tools we need to stop the novel coronavirus and safely re-open the economy. But they are on the horizon.https://gatesnot.es/2S2v7g9\xa0",
 'Bill Gates Retweeted Code.org',
 'This is a great introduction to computer science concepts for students at all levels. I’m looking forward to joining today’s Code Break.https://twitter.com/codeorg/status/1251143713402662912\xa0…',
 'Bill Gates added,',
 'Halting funding for the World Health Organization during a world health crisis is as dangerous as it sounds. Their work is slowing the spread of COVID-19 and if that work is stopped no other organization can replace them. The world needs @WHO now more than ever.',
 'Bill Gates Retweeted Nic

In [13]:
ParagraphsDF = pd.DataFrame({'paragraph-text' : Paragraphs})
print(ParagraphsDF)

                                       paragraph-text
0   We've detected that JavaScript is disabled in ...
1                                                 Yes
2   Sharing things I'm learning through my foundat...
3   Are you sure you want to view these Tweets? Vi...
4   We don't yet have all the tools we need to sto...
5                       Bill Gates Retweeted Code.org
6   This is a great introduction to computer scien...
7                                   Bill Gates added,
8   Halting funding for the World Health Organizat...
9               Bill Gates Retweeted Nicholas Kristof
10  .@NickKristof does an amazing job capturing th...
11                                  Bill Gates added,
12  On #WorldHealthDay — and every day — I want to...
13                Bill Gates Retweeted The Daily Show
14  I always enjoy chatting with @Trevornoah, even...
15                                  Bill Gates added,
16              Bill Gates Retweeted Gates Foundation
17  It’s encouraging to see 

In [20]:
findNumber = r'\d'
regexResults = re.search(findNumber, ParagraphsDF['paragraph-text'][6])
regexResults.group(0)

'1'

In [21]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


In [22]:
#Spidering

otherPAgeURLS = []
for paragraphNum, pTag in enumerate(bgPTags):
    tagLinks = pTag.findAll('a', href=re.compile('.twitter.'), class_=False)
    for aTag in tagLinks:
        relurl = aTag.get('href')
        linkText = aTag.text
        otherPAgeURLS.append((
            urllib.parse.urljoin(twitter_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('http://status.twitter.com', 38, 'Twitter Status'), ('http://support.twitter.com/forums/26810/entries/78525', 40, 'Learn more'), ('https://dev.twitter.com/web/embedded-tweets', 42, 'Learn more'), ('https://dev.twitter.com/web/embedded-tweets', 43, 'Learn more'), ('https://dev.twitter.com/overview/terms/agreement', 45, 'Developer Agreement'), ('https://dev.twitter.com/overview/terms/policy', 45, 'Developer Policy')]


In [23]:
print(bgPTags)

[<p>We've detected that JavaScript is disabled in your browser. Would you like to proceed to legacy Twitter?</p>, <p class="NoScriptForm-buttonContainer"><button class="EdgeButton EdgeButton--primary" type="submit">Yes</button></p>, <p class="ProfileHeaderCard-bio u-dir" dir="ltr">Sharing things I'm learning through my foundation work and other interests.</p>, <p class="ProfileWarningTimeline-explanation">Are you sure you want to view these Tweets? Viewing Tweets won't unblock <span class="username u-dir u-textTruncate" dir="ltr">@<b>BillGates</b></span></p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">We don't yet have all the tools we need to stop the novel coronavirus and safely re-open the economy. But they are on the horizon.<a class="twitter-timeline-link u-hidden" data-expanded-url="https://gatesnot.es/2S2v7g9" dir="ltr" href="https://t.co/1q7YHRXokz" rel="nofollow noopener" target="_blank" title="https://gatesnot.es

In [24]:
ParagraphsDF['source'] = [twitter_base_url+bg] * len(ParagraphsDF['paragraph-text'])
ParagraphsDF['paragraph-number'] = range(len(ParagraphsDF['paragraph-text']))

ParagraphsDF

,paragraph-text,source,paragraph-number
0,We've detected that JavaScript is disabled in ...,http://twitter.com/BillGates,0
1,Yes,http://twitter.com/BillGates,1
2,Sharing things I'm learning through my foundat...,http://twitter.com/BillGates,2
3,Are you sure you want to view these Tweets? Vi...,http://twitter.com/BillGates,3
4,We don't yet have all the tools we need to sto...,http://twitter.com/BillGates,4
5,Bill Gates Retweeted Code.org,http://twitter.com/BillGates,5
6,This is a great introduction to computer scien...,http://twitter.com/BillGates,6
7,"Bill Gates added,",http://twitter.com/BillGates,7
8,Halting funding for the World Health Organizat...,http://twitter.com/BillGates,8
9,Bill Gates Retweeted Nicholas Kristof,http://twitter.com/BillGates,9


In [26]:
ParagraphsDF['source-paragraph-number'] = [None] * len(ParagraphsDF['paragraph-text'])
ParagraphsDF['source-paragraph-text'] = [None] * len(ParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pd.DataFrame(parsDict)

for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    ParagraphsDF = ParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
ParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,We've detected that JavaScript is disabled in ...,http://twitter.com/BillGates,None,None
1,1,Yes,http://twitter.com/BillGates,None,None
2,2,Sharing things I'm learning through my foundat...,http://twitter.com/BillGates,None,None
3,3,Are you sure you want to view these Tweets? Vi...,http://twitter.com/BillGates,None,None
4,4,We don't yet have all the tools we need to sto...,http://twitter.com/BillGates,None,None
...,...,...,...,...,...
63,1,Â,http://support.twitter.com/forums/26810/entrie...,40,Learn more
64,2,Even after you enable Tweeting with your locat...,http://support.twitter.com/forums/26810/entrie...,40,Learn more
65,3,"Note: In some areas, you have the option to la...",http://support.twitter.com/forums/26810/entrie...,40,Learn more
66,4,\n By using Twitterâs services yo...,http://support.twitter.com/forums/26810/entrie...,40,Learn more


In [27]:
#Section 2 API


stdoutOrigin=sys.stdout 
sys.stdout = open("out.txt", "w")
csv_to_save = "Twitter_data_table.csv"
example_text_file = "examplefile.txt"

url = "http://api.twitter.com/1.1/search/tweets.json"
API_key = "dAcQ3xOznkH4weiHNcOCtJlBU"
API_secret = "hXLhDenPi0BXLpiEf21726R6VnIqCdwEF2Bec1If3EE4iU9TIr"
Access_token = "1219979323945443328-xk9kQNKCCtAXN3gk4pOy86YO3cgW4i"
Access_token_secret = "vnn7ylL8qHUwB27nGi7POhFidO9hjWbMDPKan2fDS2CSY"

tracklist = ['#COVID-19','#coronavirus']
tweet_count = 0
n_tweets = 500

class stdOutListener(StreamListener):
    def on_data(self,data):
        global tweet_count
        global n_tweets
        global stream
        if tweet_count < n_tweets:
            print(data)
            tweet_count += 1
            return True
        else:
            stream.disconnect()

    def on_error(self,status):
        print(status)

if __name__ == '__main__':
    l = stdOutListener()
    auth = OAuthHandler(API_key,API_secret)
    auth.set_access_token(Access_token,Access_token_secret)
    stream = Stream(auth,l)
    stream.filter(track=tracklist)

sys.stdout.close()
sys.stdout=stdoutOrigin

In [48]:
tweets_data_path = 'out.txt'
tweets_data = []
tweets_file = open(tweets_data_path,'r')
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
print(tweets_data[:2])

[{'created_at': 'Sat Apr 25 15:51:26 +0000 2020', 'id': 1254075553184956416, 'id_str': '1254075553184956416', 'text': 'News from Image Werks Corporation is out! https://t.co/qTLYOBfNQg Stories via @NatriceR #coronaviruspandemic #coronavirus', 'source': '<a href="https://paper.li" rel="nofollow">Paper.li</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 401278284, 'id_str': '401278284', 'name': 'The Image Works Corp', 'screen_name': 'Aja_7', 'location': 'East Providence, RI', 'url': 'http://imagewerkscorp.wordpress.com', 'description': 'To effect change a person must choose effective words. My content strategy words are: “Faith, Joy, Prosper, and AgapeLove!” http://www.twylah.com/Aja_7.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1314, 'friends_count': 1846, 'listed_count': 160, 'favourites_coun

In [29]:
tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet : tweet['text'],tweets_data))
tweets['Username'] = list(map(lambda tweet:tweet['user']['screen_name'],tweets_data))
tweets['Timestamp'] = list(map(lambda tweet:tweet['created_at'],tweets_data))
tweets['location'] = list(map(lambda tweet:tweet['user']['location'],tweets_data))
tweets

,text,Username,Timestamp,location
0,News from Image Werks Corporation is out! http...,Aja_7,Sat Apr 25 15:51:26 +0000 2020,"East Providence, RI"
1,RT @WalterEJLC: Il n'est pas impossible que la...,emilecabaret721,Sat Apr 25 15:51:26 +0000 2020,soutien incontestable à EJL
2,"RT @symbolik: Check out Toronto, CA Lockdown 2...",Pesos_Beatery,Sat Apr 25 15:51:26 +0000 2020,Between 20 Hz - 20k Hz
3,RT @jlsteeg: No tengo tiempo de tuitear. En @V...,MRBP2020,Sat Apr 25 15:51:26 +0000 2020,None
4,German Drosten has found indications in recent...,UnusualUpdate,Sat Apr 25 15:51:26 +0000 2020,None
...,...,...,...,...
495,RT @AFPespanol: #ULTIMAHORA No existen pruebas...,nachoespinoza_,Sat Apr 25 15:52:05 +0000 2020,None
496,Check out my Gig on Fiverr: make your photo in...,erikhermawann22,Sat Apr 25 15:52:05 +0000 2020,"Makassar , Gowa , Bone ,Sinjai"
497,RT @jlsteeg: No tengo tiempo de tuitear. En @V...,tango_luna,Sat Apr 25 15:52:05 +0000 2020,Còrdoba
498,RT @garimagarg_: Im legit in awe of China’s di...,SidGarg13,Sat Apr 25 15:52:05 +0000 2020,None


In [30]:
# Raw Text
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)
    
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



In [32]:
with open('/Users/shiyipeng/nltk_data/corpora/udhr2/007.txt') as f:
    oo = f.read()
print(oo[-500:])

     
      
        Óla qualqué nguê ca gozá dirêtu cu liberdade, selá ê cumpli lei pa plomovê lispêtu di dirêtu cu di liberdade d'ôtlo nguê, pa moral, lei di govenu e gozo geral ni democracia sá cuá vedé.
      
      
        Tudu dirêtu cu liberdade sé na ca pô zisti contla vonté di Naçón Unida.
      
    
  
    Artigo 30º
    Tudu cuá cu sá ni Declaraçón sé na ca pô plimiti qualqué Estado, glupu ô nguê di fé qualqué cuá cu ca pô plijudicá tudu dirêtu cu liberdade cu sá ni ducumento sé.
  


In [35]:
targetDir = '/Users/shiyipeng/nltk_data/corpora/udhr2' #Change this to your own directory of texts
ooText = []
ooFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        ooText.append(f.read())
        ooFileName.append(file.name)
ooFileName

['kmb.txt',
 'roh.txt',
 'ydd.txt',
 'ibo.txt',
 'urd.txt',
 'acu.txt',
 'mya.txt',
 'hsf.txt',
 'cbs.txt',
 'ton.txt',
 'cot.txt',
 'mag.txt',
 'cbr.txt',
 'ote.txt',
 'src.txt',
 'amh.txt',
 'eml.txt',
 'ita.txt',
 'ukr.txt',
 'guj.txt',
 'pau.txt',
 'nya_chinyanja.txt',
 'gax.txt',
 'kng_AO.txt',
 'cym.txt',
 'zul.txt',
 'cak.txt',
 'hrv.txt',
 'mlt.txt',
 'mad.txt',
 'mar.txt',
 'dip.txt',
 'tyv.txt',
 'tzm.txt',
 'eve.txt',
 'rus.txt',
 'ktu.txt',
 'kat.txt',
 'lin.txt',
 'rar.txt',
 'eng.txt',
 'tzm_tfng.txt',
 'ron.txt',
 'emk.txt',
 'ewe.txt',
 'ndo.txt',
 'pol.txt',
 'hin.txt',
 'jpn.txt',
 'ssw.txt',
 'tat.txt',
 'cbu.txt',
 'tuk_cyrl.txt',
 'cos.txt',
 'toi.txt',
 'shk.txt',
 'zam.txt',
 'cbt.txt',
 'srr.txt',
 'tzh.txt',
 'ace.txt',
 'bis.txt',
 'koo.txt',
 'kwi.txt',
 'ura.txt',
 'plt.txt',
 'tur.txt',
 'sey.txt',
 'cof.txt',
 'skr.txt',
 'toj.txt',
 'div.txt',
 'fri.txt',
 'por_PT.txt',
 'hsb.txt',
 'deu.txt',
 'tbz.txt',
 'ell_polytonic.txt',
 'vmw.txt',
 'mxv.txt',
 'mz

In [37]:
oo_df = pd.DataFrame({'text' : ooText}, index = ooFileName)
oo_df

,text
kmb.txt,KITANGELU KYA NGONGO YA ITOKELU YA MUTHU\n ...
roh.txt,Decleranza universala dals drets da l'uman\n ...
ydd.txt,אַלװעלטלעכע דעקלאַראַציע פֿון מענטשנרעכט\n ...
ibo.txt,NKUWUWAPỤTA ỤWA NILE BANYERE IKIKE MMADỤ NWEGA...
urd.txt,انسانی حقوق کا عالمی منشور\n اقوامِ متحدہ...
...,...
kal.txt,INUTTUT PISINNAATITAAFFIIT PILLUGIT SILARSUARM...
lia.txt,Ku Koloŋ ko ka kuloho ko ka wɔɔ yo wɔ Kafaydo\...
bem.txt,IFYASOSELE IFYALO PALWA NSAMBU SHA MUNTU\n ...
knc.txt,Hakkiwa Adamganabe Dunya Ngaro Wowurtəgənama\n...


In [45]:
information_extraction_pdf = 'https://static1.squarespace.com/static/564b414be4b07bf683d4222c/t/590762a0725e250c61187aef/1493656227799/Chinatown+Community+Research+Report_Apirl2017_rfs.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%����
xref
197 26
0000000016 00000 n
0000001727 00000 n
0000001793 00000 n
0000002100 00000 n
0000002322 00000 n
0000002538 00000 n
0000002753 00000 n
0000002799 00000 n
0000002836 00000 n
0000003267 00000 n
0000003693 00000 n
0000004384 00000 n
0000005354 00000 n
0000005545 00000 n
0000007536 00000 n
0000007744 00000 n
0000008062 00000 n
0000010772 00000 n
0000010809 00000 n
0000010861 00000 n
0000016790 00000 n
0000029714 00000 n
0000039111 00000 n
0000172201 00000 n
0000174587 00000 n
0000000816 00000 n
<</Filter/FlateDecode/I 1017/Length 828/S 738>>stream
�d�.�


In [46]:
import pdfminer
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)
print(readPDF(infoExtractionBytes)[:550])

Chinatown Anti-Displacement Community Research 

Project Report 

	

 

 

 

 

 

APRIL 2017 

 

School of Communication, Loyola University Chicago 

George Villanueva, Ph.D. 

Coalition for a Better Chinese American Community 

Debbie Liu, M.P.H 

 

Sponsors: Asian Americans Advancing Justice | Chicago MAARC (Midwest Asian American 
Research Collaboratory) and Kellogg Foundation 

 

	

2	

Table of Contents 

 

 

 

 

INTRODUCTION 
 
METHODS AND DATA COLLECTION 
 
FINDINGS 
 
DISCUSSION AND IMPLICATIONS  
 
REFERENCES   
 
APPENDIX  

